## Notebook for testing different stuff and building a dataset


In [3]:
import pandas as pd 
import json

with open('nhs_illnesses.json') as f:
    data = json.load(f)


In [4]:
data.keys() # no ilnesses for Q and X = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z']
data['A'][0] # each leter has a list of illness, link 

# dataset should include title, symptoms, document, and link 
# keep symptoms and document empty for now
# create a list of dictionaries
illnesses = []
for letter, illnesses_list in data.items():
    for illness in illnesses_list:
        illness_dict = {
            'title': illness[0],
            'symptoms': '',
            'document': '',
            'link': illness[1]
        }
        illnesses.append(illness_dict)
# create a dataframe from the list of dictionaries
df = pd.DataFrame(illnesses)


In [5]:
df.iloc[0:5]

,title,symptoms,document,link
0,Abdominal aortic aneurysm,,,https://www.nhsinform.scot/illnesses-and-condi...
1,Achilles tendinopathy,,,https://www.nhsinform.scot/illnesses-and-condi...
2,Acne,,,https://www.nhsinform.scot/illnesses-and-condi...
3,Acute cholecystitis,,,https://www.nhsinform.scot/illnesses-and-condi...
4,Acute lymphoblastic leukaemia,,,https://www.nhsinform.scot/illnesses-and-condi...


In [8]:
# use the function from link_scraper.py to scrape the data from the link into description
from link_scraper import get_relavent_text

df['document'] = df['link'].apply(lambda x: get_relavent_text(x)['content']) # apply the function to the link column


In [11]:
# sample 5 links from the data
df.sample(5)['link'].values.tolist()


['https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/conditions-that-can-affect-multiple-parts-of-the-body/polymyalgia-rheumatica/',
 'https://www.nhsinform.scot/illnesses-and-conditions/immune-system/allergies/',
 'https://www.nhsinform.scot/illnesses-and-conditions/ears-nose-and-throat/feeling-of-something-in-your-throat-globus/',
 'https://www.nhsinform.scot/illnesses-and-conditions/muscle-bone-and-joints/leg-and-foot-problems-and-conditions/achilles-tendinopathy/',
 'https://www.nhsinform.scot/illnesses-and-conditions/stomach-liver-and-gastrointestinal-tract/constipation/']

In [41]:
# function to search the docuemtn where ['heading'] contains the word 'symptoms'
def search_word_in_heading(doc, words):
    for section in doc:
        heading = section['heading']
        for word in words:
            if word in heading.lower():
                return section['text']
    return None
    
# apply the function to the document column
df['symptoms'] = df['document'].apply(lambda x: search_word_in_heading(x, ['symptom', 'sign', 'cause', 'diagnos'])) # apply the function to the document column
# check the first 5 rows
df['whole_page'] = ''
for i in range(df.shape[0]):
    webpage = df.iloc[i]['title'] + '\n\n'
    for section in df['document'].iloc[i]:
        webpage += section['heading'] + '\n' + ''.join(section['text']) + '\n\n'
    df['whole_page'].iloc[i] = webpage

df.to_json('nhs_illnesses_structured.json', orient='index') # save the dataframe to a json file
        




/var/folders/v0/131nk04n239dcq_8ftxwwy040000gn/T/ipykernel_72942/232571567.py:18: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['whole_page'].iloc[i] = webpage
/var/folders/v0/131nk04n239dcq_8ftxwwy040000gn/T/ipykernel_72942/232571567.py:

In [28]:
# get the rows where symptoms is null
df[df['symptoms'].isnull()].sample(5)


,title,symptoms,document,link
355,Sudden infant death syndrome (SIDS),None,"[{'heading': 'Sudden infant death syndrome', '...",https://www.nhsinform.scot/ready-steady-baby/e...
209,Lactose intolerance,None,"[{'heading': 'About lactose intolerance', 'tex...",https://www.nhsinform.scot/illnesses-and-condi...
329,Septic shock,None,[{'heading': 'Who is more at risk of developin...,https://www.nhsinform.scot/illnesses-and-condi...
51,Brain tumours,None,"[{'heading': 'When to speak to your GP', 'text...",https://www.nhsinform.scot/illnesses-and-condi...
180,Hives,None,"[{'heading': '', 'text': ['', 'Last Updated: '...",https://www.nhsinform.scot/illnesses-and-condi...


In [42]:
# get avarage length of the whole_page
df['whole_page'].apply(lambda x: len(x.split(' '))).mean()

np.float64(1450.9732360097323)

In [ ]:
string = 'this is a string with a value {0}'
string.format(5) # format the string with a value